# 1. the sigmoid proof

Show that the derivative of the sigmoid function given above is given by:

    𝜕𝜎(𝑥)
𝜕𝑥 = 𝜎(𝑥)(1 − 𝜎(𝑥))

tests
## remebering the sigmoid function:
$$ 
\partial\sigma(x)= \partial(\frac{1}{1+e^{-x}})
$$

## remembering the formula for quotients
$$
\frac{dy}{dx}\left(\frac{f(x)}{f(g)}\right) 
=
\frac{\frac{d}{dx}\left(f(x)\right)  g(x) -  f(X)\frac{d}{dx}\left(g(x)\right) }  {[g(x)]^2}
$$

## breaking it into peices for sigmoid
$$
f'(x) = 0 \newline
g'(x) = -e^{-x}
$$
## spelling it out 
$$
\frac{0(1+e^{-x})-1(-e^{-x})            }
     {\left(1+e^{-x}\right)^2}
$$
## result

$$
\frac{e^{-x}}
     {\left(1+e^{-x}\right)^2} 
$$

## below we will verify the result

In [1]:
from sympy import E, diff 
from sympy.abc import x
import numpy as np
expr = 1/(1+E**-x)
#diff(a*log(a) - a + 1, x)
expr

1/(1 + exp(-x))

In [2]:
diff(expr)

exp(-x)/(1 + exp(-x))**2

We have verified our differentiation but I am still unsure how to get it into the requested form.


# 2. testing forward propegartion to predicts
Here are the initial weights:
WEIGHTS  

| Link from node | Link to node | Weight |
|----------------|--------------|--------|
| Input 1        | Hidden 1     | 0.1    |
| Input 1        | Hidden 2     | 0.3    |
| Input 1        | Hidden 3     | −0.2   |
| Input 2        | Hidden 1     | −0.4   |
| Input 2        | Hidden 2     | 0.1    |
| Input 2        | Hidden 3     | 0.2    |
| Input 3        | Hidden 1     | −0.1   |
| Input 3        | Hidden 2     | −0.2   |
| Input 3        | Hidden 3     | 0.4    |
| Hidden 1       | Out 1        | 0.5    |
| Hidden 1       | Out 2        | 0.2    |
| Hidden 2       | Out 1        | −0.3   |
| Hidden 2       | Out 2        | −0.3   |
| Hidden 3       | Out 1        | 0.2    |
| Hidden 3       | Out 2        | 0.1    |
|                | BIAS WEIGHTS |        |
|                | Layer        | Weight |
|                | Hidden       | 0.2    |
|                | Output       | 0.3    |


**steps** 
using these weights we will calculate a prediction and error for the following input  



| input 1 | input 2 | input 3 | Output 1 | Output 2 |
|---------|---------|---------|----------|----------|
| 4       | 8       | 6       | .65      | .4       |



## populating our first weight matrix and input

In [3]:

#to input -> hiden knode weights 
w =np.array([[.1,.3,-.2],[-.4,.1,.2],[-.1,-.2,.4],[.2,.2,.2]])
print(w.shape)
print(w)
inp = np.array([4,8,6,1])
inp


(4, 3)
[[ 0.1  0.3 -0.2]
 [-0.4  0.1  0.2]
 [-0.1 -0.2  0.4]
 [ 0.2  0.2  0.2]]


array([4, 8, 6, 1])

## calculating our weighted sums

In [4]:
whid=inp.dot(w)
whid

array([-3.2,  1. ,  3.4])

In [5]:
#sanity checking
4*.1 + 8*-.4 + 6*-.1 + .2

-3.2

## transformating hidden node with sigmoid

In [6]:
#creating a sigmoid function sigmoid
import numpy as np

def sigmoid(x):
    return 1/(1+np.e**(-x))

In [7]:
#applying the sigmoid 
Thid = np.append(sigmoid(whid),1)
Thid.shape

(4,)

## populating our second weight table.

In [8]:
#to out from hidden node weights
W =np.array([[.5,.2],[-.3,-.3],[.2,.1],[.3,.3]])
W

array([[ 0.5,  0.2],
       [-0.3, -0.3],
       [ 0.2,  0.1],
       [ 0.3,  0.3]])

## finding the weighted sum of outputs

In [9]:
#calculating Weighted output
Wout=Thid.dot(W)
Wout

array([0.29380619, 0.18528602])

In [10]:
## applying the sigmoid and getting our outputs
Tout = sigmoid(Wout)
Tout

array([0.5729277 , 0.54618944])

## now putting it all together into one function 

In [11]:
sigmoid(np.append(sigmoid(inp.dot(w)),1).dot(W))

array([0.5729277 , 0.54618944])

In [33]:
# building a function for later:
def predict(inp,w,W,func=sigmoid):
    return func(np.append(func(inp.dot(w)),1).dot(W))
predict(inp,w,W)

array([0.5729277 , 0.54618944])

In [13]:
targ = np.array([.65,.4])
targ

array([0.65, 0.4 ])

In [14]:
E = (sum(targ - Tout)**2)*.5
E

0.0023885891017721714

In [15]:
def sigmoid(x):
    """our sigmoid activation funciton """
    import numpy as np #importing dependencies as is best practice 
    return 1/(1+np.e**(-x))


def error(targ, pred):
    """ error function"""
    return  (sum(targ - pred)**2)*.5

def predict(inp,w,W,func=sigmoid, y=None):
    """this function takes in inputs, weight matrix, and an activation function and makes a predication """
    pred = func(np.append(func(inp.dot(w)),1).dot(W))
    if E is None: 
        return pred
    else: 
        
        return error(y, pred)

   
    


In [16]:
predict(inp,w,W,y=targ)

0.0023885891017721714

# 3.working on backware propegaation:

**our back propegation several steps:**
1. find and store our gradients:  
   As we are trying to minimize our errors the gradients we find are for the funciton for are from each weight and bias to the Error term.  
    1. first step find gradients for output to hidden
        1. for all hidden nodes and output nodes Gradiants for
        2. for BiasO for all output nodes
    3. second step find gradients for hidden to input
        1. for all hidden nodes and inputs 
        2. for BiasH for all hidden nodes 
    4.  use calcualted gradients to apply update to weights
        

**functions and informaiton we need to power our gradients:**

* input information: inputs, target outputs, weight matrix's 
* cacluated iformaiton: 
    * Thid - our activated hiden nodes values  with respect to our input 
    * Tout - our predictations with respect to our input 
* funcs"
    * derivit funciton of our error function with respect to E 
    * derivitive funciton of  our activatiion function  


**Detailed steps:**
1. initalize matrix's g and G of shape IxH and HxO where I is Num_inputs, H is Num_hideen nodes, and O is num_ outputs 
2. initializ 2 bias arrays if len H and O 
3. use a nested for loop to cacluate for all weights between all hidden and output noodes the gradient with resepect to each weight:
    3. first caculate the Bias term for kth output node
    4. use the bias term for k and multipy by the jth hidden node to  to find the gradeints each hidden node j output node k 
    5. store the biask for use in later caculatations
        * special care has to be taken here biases must be summed before being utlized in gradiant updates
4. use a nested for loop to calculate for all weights between  all input and hidden noedes the gradient with repect to each weight:
    1. for each hidden node j and bias k from the previous step find the dot product between weight vector and biases
    2. find the bias for each hidden node j
    3. use the bias for hidden node j and multiply for each input i to find the gradient for each weight i,j 
    4. update Biash with bias term
5. update gradient matrixs
    1. for hidden to output gradient matrix
        1. sum bias vector and then appened to matrix
    2. for input to hiddne gradient matrix
        1. append bias vector 
5. apply gradients to weights using formula $weight-\alpha*gradient$ we will use alpha .2
 

## initalizing

In [17]:
num_inputs = 3
num_hidden = 3
num_outputs = 2

# intialize matrix and vectors
G = np.zeros((num_hidden,num_outputs))
g = np.zeros((num_inputs,num_hidden))
Bo = np.zeros((1,num_outputs))
Bh = np.zeros((1,num_hidden))

In [18]:
# set up our derivitive error function
def Der_error(pred,target):
    return pred-target
# set up our derivitive activation function
def Der_sigmoid(val):
    return val *(1-val)

## first step back

In [19]:
#3.loop and cacluate gradiants and  bias for first step back 


for k in range(num_outputs):
    #caculate for an output node 
    Bias = Der_error(Tout[k],targ[k])*Der_sigmoid(Tout[k])
    for j in range(num_hidden):
        #add for hidden node 
        G[j,k]=Bias*Thid[j]
    
    #upate bias vectore
    Bo[0,k] = Bias #note this will be summed before utlized in updates but information stored here can be utlized in next step
    

## second step back

In [20]:
#loop and caculate gradients and  bias for second step back
for k in range(num_hidden):
    bias = Bo.dot(W[k,:])*Der_sigmoid(Thid[k])
    for j in range(num_inputs):
        g[j,k] = bias*inp[j]
    Bh += bias #updating all biaas because we aren't using these values again

##   

In [21]:
# clean up gradient arrays 

#sum Bo
Bo[:] = np.sum(Bo)
G = np.append(G,Bo,axis=0)
g = np.append(g,Bh,axis=0)


In [22]:
#update weights 

W_new = W-.2*G
w_new = w-.2*g

In [23]:
w_new

array([[ 0.10006569,  0.30081998, -0.1999963 ],
       [-0.39986862,  0.10163996,  0.2000074 ],
       [-0.09990147, -0.19877003,  0.40000555],
       [ 0.20022234,  0.20022234,  0.20022234]])

# 4. bulding everything together

we will now take all the steps above a function.

In [39]:
num_inputs = 3
num_hidden = 3
num_outputs = 2
def get_gradients(x, y, alpha =.2, batch=False):
    # initalize
    G = np.zeros((num_hidden, num_outputs))
    g = np.zeros((num_inputs, num_hidden))
    Bo = np.zeros((1,num_outputs))
    Bh = np.zeros((1,num_hidden))

    #pulling weights from global scope
    global W
    global w

    # create predictions
    cur_pred = predict(x, w=w, W=W)

    # first step loop
    for k in range(num_outputs):
        # calculate for an output node
        pBias = Der_error(cur_pred[k],y[k]) *  Der_sigmoid(cur_pred[k])
        for j in range(num_hidden):
            # add for hidden node
            G[j, k] = pBias*Thid[j]

        # update bias vector
        Bo[0, k] = pBias     # note this will be summed before utlized in updates but information stored
        # here can be utlized in next step

    # second step loop
    for k in range(num_hidden):
        pbias = Bo.dot(W[k,:]) * Der_sigmoid(Thid[k])
        for j in range(num_inputs):
            g[j, k] = pbias*x[j]
        Bh += pbias  # updating all bias because we aren't using these values again

    # clean up gradient arrays and append bias's
    Bo[:] = np.sum(Bo)
    G = np.append(G,Bo,axis=0)
    g = np.append(g,Bh,axis=0)

    if batch is True:
        print("one pass")
        return g, G
    elif batch is False:

        W = W - alpha*G

        w = w - alpha*g
        return print('one pass weights updated')

    else:
        return ('bad options for batch should be True/False')

In [41]:
get_gradients(inp,targ, batch=True)

one pass


(array([[-3.28448805e-04, -4.09990114e-03, -1.85123644e-05],
        [-6.56897609e-04, -8.19980228e-03, -3.70247289e-05],
        [-4.92673207e-04, -6.14985171e-03, -2.77685467e-05],
        [-1.11171558e-03, -1.11171558e-03, -1.11171558e-03]]),
 array([[-0.00073859,  0.00141919],
        [-0.01378643,  0.02649025],
        [-0.01824914,  0.03506523],
        [ 0.0173773 ,  0.0173773 ]]))

In [51]:
inp_many = np.tile(inp,3).reshape((-1,4))

targ_many = np.tile(targ,3).reshape((-1,2))

In [61]:
for x, y in zip(inp,targ):
    print(error(y,predict(x,w,W)))

AttributeError: 'numpy.int32' object has no attribute 'dot'